# Solving Quintics

In this section, we explain how to exploit the icosahedral symmetry discussed before to solve an arbitrary polynomial of order five (also known as a quintic polynomial).

We begin by returning to a result derived earlier: in the [Section on Icosahedral Symmetry](icosa), we showed that the the icosahedral face centres can be grouped into five octohedra:

$$

T(u,v) = \prod_{k=1}^5 t_I(u,v, \epsilon^k) = u^{30} +522\left(u^{25}\,v^5-u^5\,v^{25}\right) - 10005\left(u^{20}\,v^{10} + u^{10}\,v^{20}\right) + v^{30} \,.


$$

where

$$

t_I(u,v, e^{i\theta}) := u^6 + 2\,u^5\,v\,e^{i\theta}-5\,u^4\,v^2\,e^{2\,i\theta} 
- 5\,u^2\,v^4\,e^{4\,i\theta} - 2\,u\,v^5\,e^{5\,i\theta} + v^6\,e^{6\,i\theta}

$$

is the polynomial invariant for a single octahedron tilted by polar angles $(\alpha, \theta)$ &mdash; once we have set $\tan 2\alpha = 2$ for an edge centre &mdash; and $\epsilon := \exp\left(2\pi i /5\right)$.

In [3]:
# Check of the above
from utils import e_simp, reduce_multiply
from sympy import symbols, expand

u, v, e = symbols("u v e")


def tI(u, v, p):
    return (
        u**6
        + 2 * u**5 * v * p
        - 5 * u**4 * v**2 * p**2
        - 5 * u**2 * v**4 * p**4
        - 2 * u * v**5 * p**5
        + v**6 * p**6
    )


e_simp(expand(reduce_multiply([tI(u, v, e**k) for k in range(5)])), e)

u**30 + 522*u**25*v**5 - 10005*u**20*v**10 - 10005*u**10*v**20 - 522*u**5*v**25 + v**30